In [1]:
from matplotlib import pyplot as plt
from utils.utils import *
import utils.promethee_functions as pf
import utils.clustering_functions as cf

data = read_data()

group0 = ["PAK", "SDN", "BDI", "HTI"]
group1 = ["EST", "CZE", "MLT", "SGP", "IRL"]
group2 = ["CHE", "ISL", "NZL", "SWE"]

all_groups = group0 + group1 + group2

data = data.loc[all_groups]

# Nb of criteria
K = data.columns.shape[0] # Nb of criteria
L = data.iloc[0]["co2prod"].shape[0] # Length of the time series

W = [1/K for i in range(K)] # Weights of the criteria (equal weights)
Q = [10, 0.3, 40,  0.4, 4, 2] # Indifference thresholds
P = [28, 0.9, 80, 1, 18, 12] # Preference thresholds

data.head(3)



utils.py Loading
Reading HDI dataset
co2prod: min=0.0, max=33.3863
hdi: min=0.257, max=0.967
le: min=37.105, max=85.473
gdi: min=0.383, max=1.041
eys: min=3.5751, max=23.2477
mys: min=1.4606, max=14.2559


,co2prod,hdi,le,gdi,eys,mys
iso3,,,,,,
PAK,"[32.817923848999996, 32.839545195, 32.81713493...","[0.394, 0.399, 0.403, 0.405, 0.409, 0.411, 0.4...","[60.066, 60.259, 60.116, 59.934, 60.116, 59.87...","[0.539, 0.57, 0.576, 0.585, 0.598, 0.589, 0.60...","[4.238239765, 4.280649002, 4.323058238, 4.3654...","[2.291861259, 2.390371085, 2.488880911, 2.5873..."
SDN,"[33.179499414999995, 33.197418881, 33.22244834...","[0.322, 0.328, 0.318, 0.328, 0.371, 0.361, 0.3...","[49.732, 50.213, 45.891, 46.575, 56.644, 51.99...","[0.755, 0.76, 0.795, 0.805, 0.762, 0.794, 0.80...","[3.575123192, 3.716464477, 3.863393641, 4.0161...","[1.5, 1.594, 1.688, 1.782, 1.876, 1.97, 2.064,..."
BDI,"[33.367526346999995, 33.363043413999996, 33.36...","[0.285, 0.287, 0.284, 0.257, 0.288, 0.284, 0.2...","[44.86, 44.227, 43.895, 37.105, 44.08, 44.059,...","[0.822, 0.824, 0.814, 0.832, 0.823, 0.829, 0.8...","[4.218900204, 4.333489895, 4.203050137, 4.4798...","[1.460590005, 1.514265001, 1.567939997, 1.6216..."


In [9]:
alternatives = data.index
columns = data.columns
phi_c_all = pf.get_all_Phi_c(data, P, Q, L)

# Phi_c_all is the score for each criteria for each alternative
# We can use this to cluster the alternatives
# We will use the KMeans algorithm to cluster the alternatives
n_clusters = 3

def K_means_static(data, k):
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=k, random_state=0).fit(data)
    return kmeans.labels_

c:\Users\raphu\Documents\ULB\MA2\ulb-master-thesis\code\src\utils\promethee_functions.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  d = a_i[c] - a_j[c]


In [15]:
phi_c_all[1].shape

(13, 33)

In [19]:
phi_c_all[0][1].shape

(33,)

In [21]:
#phi_c_all is an array of k x n x l
# k is the number of criteria
# n is the number of alternatives
# l is the length of the time series

# I want to reshape it as a dataframe
# Each row is an alternative
# Each column is a criteria
# Each cell is a time series

#phi_c_all[0] is a list of n time series of length l, i want each row to be a time series
df = pd.DataFrame()

for i in range(len(columns)):
    df[columns[i]] = phi_c_all[i].tolist()
    df.index = alternatives

df

,co2prod,hdi,le,gdi,eys,mys
iso3,,,,,,
PAK,"[0.1022802583657407, 0.08674796937500004, 0.05...","[-0.11055555555555555, -0.10847222222222216, -...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.024027777777777707, -0.007500000000000007,...","[-0.2757193976488095, -0.28163654167857144, -0...","[-0.33911842201666664, -0.34510410224999993, -..."
SDN,"[0.1073021412268518, 0.09171843723611112, 0.06...","[-0.20055555555555554, -0.1972222222222222, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.008...","[-0.3137939139345238, -0.3141535945654761, -0....","[-0.39850801644166667, -0.404831933625, -0.412..."
BDI,"[0.10991362639351845, 0.09401877797222222, 0.0...","[-0.2468055555555556, -0.24847222222222223, -0...","[0.0, 0.0, 0.0, -0.007881250000000046, 0.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.010...","[-0.2767554455595238, -0.27880577955357144, -0...","[-0.40146376606666667, -0.41081205854999997, -..."
HTI,"[0.10844709181018516, 0.09269131838888889, 0.0...","[-0.05388888888888889, -0.05430555555555555, -...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.011...","[-0.07145364388095243, -0.07512224044047622, -...","[-0.3099937615666667, -0.315745576875, -0.3229..."
EST,"[-0.4094202551990739, -0.3342523455046297, -0....","[0.04472222222222222, 0.04083333333333332, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.009305555555555548, 0.0050000000000000044, ...","[0.0969289504047619, 0.08744647350595237, 0.08...","[0.21526125125000006, 0.22118250849166665, 0.2..."
CZE,"[-0.10547973290740736, -0.07798573876851853, -...","[0.048611111111111105, 0.04513888888888888, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.023...","[0.07361229445238093, 0.06984802223809523, 0.0...","[0.18301653869166662, 0.18546371564999992, 0.1..."
MLT,"[0.03176991480555555, 0.026150039476851853, 0....","[0.03847222222222221, 0.038333333333333316, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.016...","[0.08429371921428572, 0.08687386017261903, 0.0...","[0.010216340799999961, 0.008527104566666643, 0..."
SGP,"[-0.07696916401851846, -0.07289787321296298, -...","[0.0663888888888889, 0.06847222222222224, 0.07...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.001944444444444431, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.13967268635714286, 0.1388443137440476, 0.13...","[0.0017052837166666433, 0.01570016998333333, 0..."
IRL,"[0.018765707476851846, 0.010196855796296292, 0...","[0.04583333333333332, 0.04680555555555555, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0018055555555555416, 0.0, 0.0, 0.0, 0.0, 0....","[0.08737967326190478, 0.08649838517261904, 0.0...","[0.05441826438333334, 0.06424833598333335, 0.0..."


In [ ]:
# Each cell being a time series, I want to get only the first value of each time series
df_0 = df.applymap(lambda x: x[0])

C:\Users\raphu\AppData\Local\Temp\ipykernel_24364\3154129141.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_0 = df.applymap(lambda x: x[0])


In [34]:
def KMeans_static_seed(data, k, seed, maxit):
    """ 
        KMeans algorithm with a forced seed
        data: the data to cluster (a pd.DataFrame of n alternatives evaluated on m criteria)
        k: the number of clusters
        seed: the initial centroids (index of them in the data)
        maxit: the maximum number of iterations
    """
    
    # Initial centroid 
    centroids = seed
    centroids = data.loc[centroids].values
    converged = False
    it = 0

    old_clusters = None
    clusters = None

    while not converged or it < maxit:
        print("it", it, "centroids", centroids)
        print("it", it, "clusters", clusters)
        # Assign each point to the closest centroid
        old_clusters = clusters
        for i in range(data.shape[0]):
            distances = np.linalg.norm(centroids - data.iloc[i].values, axis=1)
            clusters[i] = np.argmin(distances)

        # Update the centroids
        for i in range(k):
            centroids[i] = np.mean(data[clusters == i], axis=0)

        if np.array_equal(old_clusters, clusters):
            converged = True
        it += 1

    return clusters

# Seed
seed = ["PAK", "EST", "CHE"]

# Get the data
data = df_0
KMeans_static_seed(data, n_clusters, seed, 1000)



it 0 centroids [[ 0.10228026 -0.11055556  0.         -0.02402778 -0.2757194  -0.33911842]
 [-0.40942026  0.04472222  0.          0.00930556  0.09692895  0.21526125]
 [ 0.0320642   0.10527778  0.          0.          0.10684037  0.26648025]]
it 0 clusters None


TypeError: 'NoneType' object does not support item assignment

In [ ]:
def get_KMeans_clusters_evolution(data, P, Q, K, L, k=3, prototype_method="seed", seed=None):
    """ 
    Get the evolution of the clusters over time using the K-Means algorithm with euclidean distance
    - data: Dataframe with the alternatives
    - P: Preference thresholds
    - Q: Indifference thresholds
    - K: Number of criteria
    - L: Length of the time series
    - k: Number of clusters to form (for the K-Medoids algorithm)
    """
    def get_dist_matrix(data, phi_c_all):
        """ Get the distance matrix between the alternatives """
        dist_matrix = np.zeros((data.shape[0], data.shape[0]))
        for i in range(data.shape[0]):
            for j in range(data.shape[0]):
                dist_matrix[i,j] = np.linalg.norm(phi_c_all[i] - phi_c_all[j])
        return dist_matrix

    alternatives = data.index
    # Get the temporal_eta matrix
    phi_c_all = pf.get_all_Phi_c(data, P, Q, L)
    dist_matrix = get_dist_matrix(data, phi_c_all)

    # Get the clusters evolution
    clusters_evolution = []
    medoids_evolution = []
    iterations = []

    for t in tqdm(range(L)):
        eta_matrix = temporal_eta[:,:,t]
        eta_matrix = pd.DataFrame(eta_matrix, index=alternatives, columns=alternatives)
        medoids, clusters, _ = cf.K_Medoid_Eta(alternatives, eta_matrix, k, prototype_method=prototype_method, print_results=False, seed=seed)
        clusters_evolution.append(clusters)
        medoids_evolution.append(medoids)
        iterations.append(t)

    return clusters_evolution, medoids_evolution, iterations


def transform_clusters_evolutions(clusters_evolution):
    """
    Transform the clusters evolution into a dataframe
    - clusters_evolution: List of the clusters evolution over time
    """
    evol = []
    for i in clusters_evolution:
        temp = []
        for key, value in i.items():
            temp.append(value)
        evol.append(temp)
    return pd.DataFrame(evol)